# iCompare Demo on select asteroids; JPL Horizon vs OpenOrb, OrbFit
## Assumptions/ documentation:

* Values for orbit definition taken from JPL Horizons elements
* Observing Location: I11, Gemini South
* Integration Time: 2010/01/01 to 2020/01/01, midnights

## Current issues:
* OrbFit 60 arcsecond bug
* OrbFit Covariance matrix effect/ workaround bug
* Find_Orb takes state vectors, also not available in COM, not available for integration in package

## Imports and Initialization of package

In [1]:
%load_ext autoreload
%autoreload 2
import icompare as ic
import pandas as pd

## Example results with default asteroids: 
### Asteroid 202930

In [2]:
# initialization for example 1
obj_id = '202930'
start = '2010-01-01T00:00:00'
stop = '2020-01-01T00:00:01'
obs = 'I11'

In [3]:
el_jpl, results = ic.get_ephems(obj_id, start, stop, obs)

Elements done
JPL Horizons done
OpenOrb done
OrbFit done


1       53.91
2       32.62
3        6.69
4       36.05
        ...  
3648    56.16
3649     8.19
3650    15.92
3651    19.34
3652    18.44
Name: 7, Length: 3653, dtype: float64', which is not in range [0,60). Treating as 0 sec, +1 min [astropy.coordinates.angle_utilities]


In [4]:
rsep, rmag = ic.gc_dist(results, reference="JPL Horizons")

In [5]:
ic.table_stats(rsep)

integrator        
OpenOrb     mean      0.041671
            median    0.025782
            max       0.154741
OrbFit      mean      0.025461
            median    0.021910
            max       0.082840
dtype: float64

### Default asteroids
202930, 110, 887, 6489, 176014, 136199, 15760, 2018VP1, 101955, 99942, 5261, 624, 588, 10199, 136472, 136199, 2008TC3, 2020VT4, 433, 367943, 2020AV2, A/2017 U1

In [6]:
result = ic.calc_all()

Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done


1       26.21
2       21.23
3       10.25
4       53.30
        ...  
3648    24.12
3649     1.42
3650    36.01
3651     7.90
3652    37.10
Name: 7, Length: 3653, dtype: float64', which is not in range [0,60). Treating as 0 sec, +1 min [astropy.coordinates.angle_utilities]


OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done


1       25.28
2        7.08
3       33.52
4       44.53
        ...  
3648     0.64
3649    20.43
3650    21.38
3651     3.59
3652    27.19
Name: 7, Length: 3653, dtype: float64', which is not in range [0,60). Treating as 0 sec, +1 min [astropy.coordinates.angle_utilities]


OrbFit done
Elements done
JPL Horizons done
OpenOrb done


...

OrbFit done
Elements done
JPL Horizons done
OpenOrb done


1       42.11
2       14.98
3       48.25
4       21.91
        ...  
3648    54.10
3649    18.41
3650    43.25
3651     8.61
3652    34.48
Name: 7, Length: 3653, dtype: float64', which is not in range [0,60). Treating as 0 sec, +1 min [astropy.coordinates.angle_utilities]


OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done


1       31.71
2       38.45
3       32.44
4       13.69
        ...  
3648    54.63
3649    19.03
3650    33.24
3651    37.21
3652    30.87
Name: 7, Length: 3653, dtype: float64', which is not in range [0,60). Treating as 0 sec, +1 min [astropy.coordinates.angle_utilities]


OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done
Elements done
JPL Horizons done
OpenOrb done
OrbFit done


## Colour Map options
The following colour/value matches have been adopted for colour-coding:

### Default colour map
value <= 0.05" -> green (good)

value <= 0.2" -> yellow (ok)

value <= 0.6" -> orange (not good but may work)

otherwise -> red (bad)

### Green/red colourblind-friendly colour map
value <= 0.05" -> no colour (good)

value <= 0.2" -> blue (ok)

value <= 0.6" -> orange (not good but may work)

otherwise -> red (bad)

In [7]:
display(pd.DataFrame(result).T.style.apply(ic.default_map))

In [8]:
display(pd.DataFrame(result).T.style.apply(ic.rg_friendly_map))